### Les fonctions stockées (Stored Procedures & Functions)

Une fonction stockée est un bloc de code SQL enregistré dans la base qui peut être exécuté plusieurs fois.  

🔹 Avantages :
- ✅ Automatiser des traitements complexes
- ✅ Réduire le trafic entre l’application et la base
- ✅ Améliorer les performances

### Création d’une fonction stockée
Une fonction stockée retourne une valeur et peut être utilisée dans une requête.

- Calculer l’âge d’un utilisateur à partir de sa date de naissance

```sql
CREATE OR REPLACE FUNCTION calculer_age(date_naissance DATE) 
RETURNS INT AS $$
BEGIN
    RETURN EXTRACT(YEAR FROM AGE(date_naissance));
END;
$$ LANGUAGE plpgsql;
```

🔹 Explication :

- ```plpgsql``` est le langage de procédure SQL de PostgreSQL
- ```EXTRACT(YEAR FROM AGE(date_naissance))``` calcule l’âge à partir de la date de naissance


- Utilisation de la fonction :

```sql
SELECT nom, calculer_age(date_naissance) AS age FROM utilisateurs;
```

### Création d’une procédure stockée

Contrairement aux fonctions, une procédure stockée ne retourne pas de valeur mais peut exécuter des actions (INSERT, UPDATE, DELETE).

- Ajouter un nouvel utilisateur et afficher un message

```sql
CREATE OR REPLACE PROCEDURE ajouter_utilisateur(nom VARCHAR, email VARCHAR, age INT) 
LANGUAGE plpgsql 
AS $$
BEGIN
    INSERT INTO utilisateurs(nom, email, age) VALUES (nom, email, age);
    RAISE NOTICE 'Utilisateur % ajouté avec succès', nom;
END;
$$;
```

- Appel de la procédure :

```sql
CALL ajouter_utilisateur('Alice', 'alice@example.com', 30);
```

### Les triggers (Déclencheurs)

Un trigger permet d’exécuter une action automatiquement avant ou après un événement (INSERT, UPDATE, DELETE).  

🔹 Cas d’usage :
- ✅ Mise à jour automatique d’un champ
- ✅ Vérification des données avant un INSERT
- ✅ Journalisation des modifications

### Création d’un trigger simple

Imaginons une table historique_commandes qui stocke les commandes supprimées.

- Création de la table d’historique

```sql
CREATE TABLE historique_commandes (
    id SERIAL PRIMARY KEY,
    commande_id INT,
    utilisateur_id INT,
    produit VARCHAR(255),
    prix DECIMAL,
    date_suppression TIMESTAMP DEFAULT now()
);
```

- Création d’une fonction pour enregistrer les commandes supprimées

```sql
CREATE OR REPLACE FUNCTION archiver_commande() 
RETURNS TRIGGER AS $$
BEGIN
    INSERT INTO historique_commandes(commande_id, utilisateur_id, produit, prix)
    VALUES (OLD.id, OLD.utilisateur_id, OLD.produit, OLD.prix);
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;
```

- Création du trigger qui déclenche la fonction après une suppression

```sql
CREATE TRIGGER trigger_archiver_commande
AFTER DELETE ON commandes
FOR EACH ROW
EXECUTE FUNCTION archiver_commande();
```

- Testons le trigger :

```sql
DELETE FROM commandes WHERE id = 1;
SELECT * FROM historique_commandes;
```

= La commande supprimée est bien enregistrée dans l’historique.